In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, partial_trace, DensityMatrix
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

![Q8](../Images/01.q8.png) 

In [2]:
basis_labels = ['|000⟩', '|001⟩', '|010⟩', '|011⟩', '|100⟩', '|101⟩', '|110⟩', '|111⟩']

# Create individual qubits and their circuits
q0_reg = QuantumRegister(1, "q0")
q1_reg = QuantumRegister(1, "q1") 
q2_reg = QuantumRegister(1, "q2")

# Create |0⟩ state (default)
circuit_0 = QuantumCircuit(q0_reg)
state_0 = Statevector.from_instruction(circuit_0)

# Create |1⟩ state
circuit_1 = QuantumCircuit(q0_reg)
circuit_1.x(q0_reg[0])  # Apply X gate to flip |0⟩ to |1⟩
state_1 = Statevector.from_instruction(circuit_1)

# Create |+⟩ state = (|0⟩ + |1⟩)/√2
circuit_plus = QuantumCircuit(q1_reg)
circuit_plus.h(q1_reg[0])  # Apply Hadamard gate
state_plus = Statevector.from_instruction(circuit_plus)

# Create |-⟩ state = (|0⟩ - |1⟩)/√2
circuit_minus = QuantumCircuit(q2_reg)
circuit_minus.x(q2_reg[0])  # First flip to |1⟩
circuit_minus.h(q2_reg[0])  # Then apply Hadamard: H|1⟩ = (|0⟩ - |1⟩)/√2
state_minus = Statevector.from_instruction(circuit_minus)

# Term 1: |0⟩ ⊗ |+⟩ ⊗ |-⟩
print("\n📍 Term 1: |0⟩|+⟩|-⟩")
term1 = state_0.tensor(state_plus).tensor(state_minus)
print("Coefficients:", term1.data)

# Term 2: |1⟩ ⊗ |0⟩ ⊗ |0⟩
print("\n📍 Term 2: |1⟩|0⟩|0⟩")
term2 = state_1.tensor(state_0).tensor(state_0)
print("Coefficients:", term2.data)

# Term 3: |1⟩ ⊗ |1⟩ ⊗ |1⟩
print("\n📍 Term 3: |1⟩|1⟩|1⟩")
term3 = state_1.tensor(state_1).tensor(state_1)
print("Coefficients:", term3.data)

# Combine: term1 + term2 - term3
print("\n📍 Step 1: Add term1 + term2 - term3")
combined = term1.data + term2.data - term3.data
print("Unnormalized coefficients:")
for i, coeff in enumerate(combined):
    if abs(coeff) > 1e-10:
        print(f"  {basis_labels[i]}: {coeff:.6f}")

# Check normalization before dividing by √3
unnorm_norm = np.sum(np.abs(combined)**2)
print(f"\nNorm before normalization: {np.sqrt(unnorm_norm):.6f}")
print(f"This should be √3 = {np.sqrt(3):.6f}")

# Normalize by √3
print("\n📍 Step 2: Normalize by √3")
psi_0_coeffs = combined / np.sqrt(3)
psi_0 = Statevector(psi_0_coeffs)

print("Final |ψ₀⟩ coefficients:")
for i, coeff in enumerate(psi_0.data):
    if abs(coeff) > 1e-10:
        print(f"  {basis_labels[i]}: {coeff:.6f}")

# Verify normalization
norm_check = np.sum(np.abs(psi_0.data)**2)
print(f"\nFinal normalization check: ||ψ₀||² = {norm_check:.6f}")

print("\n" + "="*60)
print("ANALYZING MIDDLE QUBIT MEASUREMENT")
print("="*60)

print("\nSeparating terms by middle qubit value:")

# Middle qubit = 0 terms (bit pattern: X0X)
print("\n📍 Terms where middle qubit = 0:")
middle_0_coeffs = []
middle_0_labels = []
for i, coeff in enumerate(psi_0.data):
    if abs(coeff) > 1e-10:
        # Check middle bit (bit 1 in 3-bit number)
        middle_bit = (i >> 1) & 1
        if middle_bit == 0:
            print(f"  {basis_labels[i]}: {coeff:.6f}, |coeff|² = {abs(coeff)**2:.6f}")
            middle_0_coeffs.append(abs(coeff)**2)
            middle_0_labels.append(basis_labels[i])

# Middle qubit = 1 terms (bit pattern: X1X)
print("\n📍 Terms where middle qubit = 1:")
middle_1_coeffs = []
for i, coeff in enumerate(psi_0.data):
    if abs(coeff) > 1e-10:
        middle_bit = (i >> 1) & 1
        if middle_bit == 1:
            print(f"  {basis_labels[i]}: {coeff:.6f}, |coeff|² = {abs(coeff)**2:.6f}")
            middle_1_coeffs.append(abs(coeff)**2)

# Calculate probabilities
p0 = sum(middle_0_coeffs)
p1 = sum(middle_1_coeffs)

print(f"\n📍 Probability calculations:")
print(f"p₀ = {' + '.join([f'{c:.6f}' for c in middle_0_coeffs])} = {p0:.6f}")
print(f"p₁ = {' + '.join([f'{c:.6f}' for c in middle_1_coeffs])} = {p1:.6f}")
print(f"Check: p₀ + p₁ = {p0 + p1:.6f}")

# Convert to fractions
print(f"\n📍 As simple fractions:")
# p0 should be 1/12 + 1/12 + 1/3 = 1/12 + 1/12 + 4/12 = 6/12 = 1/2
print(f"p₀ = 1/12 + 1/12 + 1/3 = 6/12 = 1/2")
print(f"p₁ = 1/12 + 1/12 + 1/3 = 6/12 = 1/2")

print("\n" + "="*60)
print("POST-MEASUREMENT STATE CALCULATION")
print("="*60)

print("\n📍 Extracting middle qubit = 0 terms:")
post_measurement = np.zeros(8, dtype=complex)
for i, coeff in enumerate(psi_0.data):
    if abs(coeff) > 1e-10:
        middle_bit = (i >> 1) & 1
        if middle_bit == 0:
            post_measurement[i] = coeff
            print(f"  {basis_labels[i]}: {coeff:.6f}")

print(f"\n📍 Normalizing by √p₀ = √{p0:.1f} = {1/np.sqrt(p0):.6f}:")
post_normalized = post_measurement / np.sqrt(p0)

print("Post-measurement state |ψ₀⟩:")
for i, coeff in enumerate(post_normalized):
    if abs(coeff) > 1e-10:
        print(f"  {basis_labels[i]}: {coeff:.6f}")

# Verify post-measurement normalization
post_norm = np.sum(np.abs(post_normalized)**2)
print(f"\nPost-measurement normalization: {post_norm:.6f}")

print("\n" + "="*50)
print("STRUCTURAL ANALYSIS")
print("="*50)

coeff_000 = post_normalized[0]  # |000⟩
coeff_001 = post_normalized[1]  # |001⟩ 
coeff_100 = post_normalized[4]  # |100⟩

print(f"\n📍 Individual coefficients:")
print(f"|000⟩: {coeff_000:.6f}")
print(f"|001⟩: {coeff_001:.6f}")
print(f"|100⟩: {coeff_100:.6f}")

print(f"\n📍 Checking |-⟩ pattern:")
print(f"|000⟩ + |001⟩ = {coeff_000 + coeff_001:.6f} (should ≈ 0)")
print(f"|000⟩ - |001⟩ = {coeff_000 - coeff_001:.6f}")

# Convert to standard form
minus_coeff = (coeff_000 - coeff_001) * np.sqrt(2)
print(f"\n📍 Standard form conversion:")
print(f"Coefficient for |0⟩|0⟩|-⟩: {minus_coeff:.6f}")
print(f"Coefficient for |1⟩|0⟩|0⟩: {coeff_100:.6f}")
print(f"Ratio: {coeff_100/minus_coeff:.6f} ≈ √2 = {np.sqrt(2):.6f}")

print(f"\n📍 Final form:")
print(f"|ψ₀⟩ = (|0⟩|0⟩|-⟩ + √2|1⟩|0⟩|0⟩) / √3")

print("\n" + "="*50)
print("VERIFICATION & ANSWER")
print("="*50)

# Verify with Qiskit
rho = DensityMatrix(psi_0)
rho_middle = partial_trace(rho, [0, 2])
p0_qiskit = np.real(rho_middle.data[0, 0])

print(f"\n📍 Qiskit verification: P(middle=0) = {p0_qiskit:.6f}")
print(f"📍 Our calculation: P(middle=0) = {p0:.6f}")
print(f"✓ Match: {abs(p0_qiskit - p0) < 1e-10}")

print(f"\n📍 FINAL ANSWER:")
print(f"p₀ = 1/2")
print(f"|ψ₀⟩ = (|0⟩|0⟩|-⟩ + √2|1⟩|0⟩|0⟩) / √3")
print(f"\n🎯 This corresponds to OPTION 2")


📍 Term 1: |0⟩|+⟩|-⟩
Coefficients: [ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j  0. +0.j -0. +0.j  0. +0.j -0. +0.j]

📍 Term 2: |1⟩|0⟩|0⟩
Coefficients: [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]

📍 Term 3: |1⟩|1⟩|1⟩
Coefficients: [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]

📍 Step 1: Add term1 + term2 - term3
Unnormalized coefficients:
  |000⟩: 0.500000+0.000000j
  |001⟩: -0.500000+0.000000j
  |010⟩: 0.500000+0.000000j
  |011⟩: -0.500000+0.000000j
  |100⟩: 1.000000+0.000000j
  |111⟩: -1.000000+0.000000j

Norm before normalization: 1.732051
This should be √3 = 1.732051

📍 Step 2: Normalize by √3
Final |ψ₀⟩ coefficients:
  |000⟩: 0.288675+0.000000j
  |001⟩: -0.288675+0.000000j
  |010⟩: 0.288675+0.000000j
  |011⟩: -0.288675+0.000000j
  |100⟩: 0.577350+0.000000j
  |111⟩: -0.577350+0.000000j

Final normalization check: ||ψ₀||² = 1.000000

ANALYZING MIDDLE QUBIT MEASUREMENT

Separating terms by middle qubit value:

📍 Terms where middle qubit = 0:
  |000⟩: 0.288675+0.0000

![q10](../Images/01.q10.png)

# Multiple Quantum Systems - Statement Analysis

## Statement 1: Bell States as Orthonormal Basis ✅ **TRUE**

**Statement:** *The Bell states form an orthonormal basis for the space corresponding to two qubits.*

The four Bell states are:
- $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$
- $|\Phi^-\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}}$
- $|\Psi^+\rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$
- $|\Psi^-\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$

**Orthogonality check:**
- $\langle\Phi^+|\Phi^-\rangle = \frac{1}{2}[\langle 00| + \langle 11|][|00\rangle - |11\rangle] = \frac{1}{2}(1 - 1) = 0$
- Similarly, all other pairs have zero inner product

**Normalization check:**
- $\langle\Phi^+|\Phi^+\rangle = \frac{1}{2}[\langle 00| + \langle 11|][|00\rangle + |11\rangle] = \frac{1}{2}(1 + 1) = 1$
- All Bell states are normalized

**Completeness:** Two-qubit Hilbert space has dimension $2^2 = 4$, and we have 4 orthonormal vectors, so they span the entire space.

---

## Statement 2: Classical Compound States ✅ **TRUE**

**Statement:** *The classical state set of a compound system is the tensor product of the classical state sets of the individual systems.*

For classical systems:
- System A: state space $\{0, 1\}$
- System B: state space $\{0, 1\}$  
- Compound system: state space $\{0, 1\} \times \{0, 1\} = \{00, 01, 10, 11\}$

This is exactly the Cartesian product (tensor product for classical sets). Each subsystem maintains its definite classical state independently.

For $n$ classical bits: $2^n$ total classical states.

---

## Statement 3: Tensor Product of Unitaries ✅ **TRUE**

**Statement:** *The tensor product of two unitary matrices represents the independent application of the two operations to two parts of a compound system.*

**Mathematical verification:**
If $U_1$ acts on subsystem 1 and $U_2$ acts on subsystem 2 independently, then:

$$(U_1 \otimes U_2)(|\psi\rangle \otimes |\phi\rangle) = (U_1|\psi\rangle) \otimes (U_2|\phi\rangle)$$

**Example:**
- $U_1 = Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$ (acts on first qubit)
- $U_2 = X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$ (acts on second qubit)

$(U_1 \otimes U_2)|01\rangle = (Z|0\rangle) \otimes (X|1\rangle) = |0\rangle \otimes |0\rangle = |00\rangle$

This represents applying $Z$ to the first qubit and $X$ to the second qubit simultaneously and independently.

---

## Statement 4: Superposition of Entangled States ❌ **FALSE**

**Statement:** *Taking a superposition of entangled states always produces an entangled state.*

**Counterexample:**
Consider two entangled Bell states:
- $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ (entangled)
- $|\Phi^-\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}}$ (entangled)

Their superposition:
$$\frac{|\Phi^+\rangle + |\Phi^-\rangle}{\sqrt{2}} = \frac{1}{\sqrt{2}}\left[\frac{|00\rangle + |11\rangle}{\sqrt{2}} + \frac{|00\rangle - |11\rangle}{\sqrt{2}}\right]$$

$$= \frac{1}{\sqrt{2}} \cdot \frac{2|00\rangle}{\sqrt{2}} = |00\rangle$$

The result $|00\rangle = |0\rangle \otimes |0\rangle$ is a **product state**, not entangled!

---

## Statement 5: Tensor Product Operation Linearity ❌ **FALSE**

**Statement:** *The operation that transforms every qubit quantum state vector $|\psi\rangle$ into the state vector $|\psi\rangle \otimes |\psi\rangle$ is a linear operation.*

**Linearity test:** For an operation $T$ to be linear: $T(\alpha|\psi_1\rangle + \beta|\psi_2\rangle) = \alpha T|\psi_1\rangle + \beta T|\psi_2\rangle$

**Left side:**
$$T(\alpha|\psi_1\rangle + \beta|\psi_2\rangle) = (\alpha|\psi_1\rangle + \beta|\psi_2\rangle) \otimes (\alpha|\psi_1\rangle + \beta|\psi_2\rangle)$$

$$= \alpha^2|\psi_1\rangle\otimes|\psi_1\rangle + \alpha\beta|\psi_1\rangle\otimes|\psi_2\rangle + \alpha\beta|\psi_2\rangle\otimes|\psi_1\rangle + \beta^2|\psi_2\rangle\otimes|\psi_2\rangle$$

**Right side:**
$$\alpha T|\psi_1\rangle + \beta T|\psi_2\rangle = \alpha|\psi_1\rangle\otimes|\psi_1\rangle + \beta|\psi_2\rangle\otimes|\psi_2\rangle$$

These are **different** due to:
1. Squared coefficients ($\alpha^2, \beta^2$ vs $\alpha, \beta$)
2. Cross terms ($|\psi_1\rangle\otimes|\psi_2\rangle + |\psi_2\rangle\otimes|\psi_1\rangle$ terms missing on right)

---

## Statement 6: Norm of Tensor Products ✅ **TRUE**

**Statement:** *The Euclidean norm of the tensor product of two vectors is equal to the product of the Euclidean norms of the two vectors.*

**Mathematical proof:**
For vectors $|\psi\rangle$ and $|\phi\rangle$:

$$\||\psi\rangle \otimes |\phi\rangle\|^2 = \langle\psi|\psi\rangle \otimes \langle\phi|\phi\rangle = \langle\psi|\psi\rangle \cdot \langle\phi|\phi\rangle = \||\psi\rangle\|^2 \cdot \||\phi\rangle\|^2$$

Therefore:
$$\||\psi\rangle \otimes |\phi\rangle\| = \||\psi\rangle\| \cdot \||\phi\rangle\|$$

This follows from the definition of the inner product in tensor product spaces and the properties of the tensor product.

---

## Statement 7: Entanglement After Measurement ❌ **FALSE**

**Statement:** *Entangled systems remain entangled when one of the systems is measured.*

**Counterexample:**
Consider the Bell state: $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$

**Measuring the first qubit:**
- **Probability of getting 0:** $P(0) = |\langle 0|_1 |\Phi^+\rangle|^2 = \frac{1}{2}$
- **Post-measurement state:** $\frac{\langle 0|_1 |\Phi^+\rangle}{\sqrt{P(0)}} = \frac{|00\rangle}{1/\sqrt{2}} = |00\rangle$

- **Probability of getting 1:** $P(1) = |\langle 1|_1 |\Phi^+\rangle|^2 = \frac{1}{2}$
- **Post-measurement state:** $\frac{\langle 1|_1 |\Phi^+\rangle}{\sqrt{P(1)}} = \frac{|11\rangle}{1/\sqrt{2}} = |11\rangle$

**Result:** Both $|00\rangle = |0\rangle \otimes |0\rangle$ and $|11\rangle = |1\rangle \otimes |1\rangle$ are **product states** (separable), not entangled.

The measurement has **destroyed the entanglement** by collapsing the quantum superposition.

---

## Summary

**True Statements:** 1, 2, 3, 6

The correct answers are:
- ✅ **Statement 1:** Bell states form orthonormal basis
- ✅ **Statement 2:** Classical compound states are tensor products  
- ✅ **Statement 3:** Tensor product represents independent operations
- ✅ **Statement 6:** Norm property of tensor products

**False Statements:** 4, 5, 7

- ❌ **Statement 4:** Superposition can create separable states
- ❌ **Statement 5:** Tensor product operation is nonlinear
- ❌ **Statement 7:** Measurement destroys entanglement

![q11](../Images/01.q11.png)

In [3]:

# Create 2-qubit circuit
q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(q)

print("📍 Step 0: Initial state |00⟩")
state = Statevector.from_instruction(circuit)
print(f"State: {state.data}")
print("This is: |00⟩")

# Step 1: Apply Z gate to bottom qubit (q[1])
print("\n📍 Step 1: Apply Z gate to bottom qubit")
circuit.z(q[1])
state = Statevector.from_instruction(circuit)
print(f"State: {state.data}")
print("Z|0⟩ = |0⟩, so state remains |00⟩")

# Step 2: Apply H gate to top qubit (q[0]) 
print("\n📍 Step 2: Apply H gate to top qubit")
circuit.h(q[0])
state = Statevector.from_instruction(circuit)
print(f"State: {state.data}")
print("This is: (|00⟩ + |10⟩)/√2")

# Step 3: Apply CNOT gate (control=q[0], target=q[1])
print("\n📍 Step 3: Apply CNOT gate")
circuit.cx(q[0], q[1])
state = Statevector.from_instruction(circuit)
print(f"State: {state.data}")
print("This is: (|00⟩ + |11⟩)/√2")

print("\n" + "=" * 40)
print("FINAL RESULT")
print("=" * 40)

# Display final state components
basis_labels = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']
print("\nFinal state components:")
for i, (label, coeff) in enumerate(zip(basis_labels, state.data)):
    if abs(coeff) > 1e-10:
        print(f"  {label}: {coeff:.6f}")

print(f"\nFinal state: |ψ⟩ = {state.data[0]:.6f}|00⟩ + {state.data[3]:.6f}|11⟩")
print(f"             = (|00⟩ + |11⟩)/√2")

print("\n📍 This is the Bell state |φ⁺⟩!")

# Compare with Bell states
print("\n" + "=" * 40)
print("BELL STATE COMPARISON")
print("=" * 40)

bell_states = {
    "|φ⁺⟩": [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)],     # (|00⟩ + |11⟩)/√2
    "|φ⁻⟩": [1/np.sqrt(2), 0, 0, -1/np.sqrt(2)],    # (|00⟩ - |11⟩)/√2  
    "|ψ⁺⟩": [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],     # (|01⟩ + |10⟩)/√2
    "|ψ⁻⟩": [0, 1/np.sqrt(2), -1/np.sqrt(2), 0]     # (|01⟩ - |10⟩)/√2
}

print("Comparing with Bell states:")
for name, bell_coeffs in bell_states.items():
    # Calculate overlap
    overlap = np.abs(np.vdot(bell_coeffs, state.data))
    print(f"{name}: overlap = {overlap:.6f}")
    if overlap > 0.99:
        print(f"  ✓ MATCH! Our circuit produces {name}")

print("\n📍 Answer: Option 1 - |φ⁺⟩ = (1/√2)|00⟩ + (1/√2)|11⟩")

# Show the circuit
print("\n" + "=" * 40)
print("CIRCUIT DIAGRAM")
print("=" * 40)
print(circuit.draw(output='text'))

📍 Step 0: Initial state |00⟩
State: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
This is: |00⟩

📍 Step 1: Apply Z gate to bottom qubit
State: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
Z|0⟩ = |0⟩, so state remains |00⟩

📍 Step 2: Apply H gate to top qubit
State: [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
This is: (|00⟩ + |10⟩)/√2

📍 Step 3: Apply CNOT gate
State: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
This is: (|00⟩ + |11⟩)/√2

FINAL RESULT

Final state components:
  |00⟩: 0.707107+0.000000j
  |11⟩: 0.707107+0.000000j

Final state: |ψ⟩ = 0.707107+0.000000j|00⟩ + 0.707107+0.000000j|11⟩
             = (|00⟩ + |11⟩)/√2

📍 This is the Bell state |φ⁺⟩!

BELL STATE COMPARISON
Comparing with Bell states:
|φ⁺⟩: overlap = 1.000000
  ✓ MATCH! Our circuit produces |φ⁺⟩
|φ⁻⟩: overlap = 0.000000
|ψ⁺⟩: overlap = 0.000000
|ψ⁻⟩: overlap = 0.000000

📍 Answer: Option 1 - |φ⁺⟩ = (1/√2)|00⟩ + (1/√2)|11⟩

CIRCUIT DIAGRAM
     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ Z ├┤ X ├
     └──

![q12](../Images/01.q12.png)

In [ ]:
# Create 2-qubit circuit
q = QuantumRegister(2, 'q')
circuit = QuantumCircuit(q)

# Initialize to |10⟩: q[0]=|1⟩, q[1]=|0⟩
circuit.x(q[0])

# Apply all operations
circuit.h(q[1])           # H on top wire
circuit.cx(q[1], q[0])    # CNOT: top controls bottom
circuit.z(q[0])           # Z on bottom wire
circuit.cx(q[0], q[1])    # CNOT: bottom controls top
circuit.h(q[0])           # H on bottom wire

# Get final state
final_state = Statevector.from_instruction(circuit)

# Print only the final result
basis_labels = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']
for i, coeff in enumerate(final_state.data):
    if abs(coeff) > 0.99:  # Find the dominant component
        print(f"Final state: {basis_labels[i]}")
        break

Final state: |11⟩
